# ConversionCentral Managed Profiling
Run this notebook from a Databricks Repo so backend deployments control profiling logic.

In [ ]:
# Collect parameters passed by the FastAPI backend


# Each widget is declared up front so Databricks jobs can safely supply overrides.
dbutils.widgets.text("table_group_id", "")
dbutils.widgets.text("profile_run_id", "")
dbutils.widgets.text("data_quality_schema", "")
dbutils.widgets.text("payload_path", "")
dbutils.widgets.text("payload_base_path", "")
dbutils.widgets.text("callback_url", "")
dbutils.widgets.text("callback_base_url", "")
dbutils.widgets.text("callback_token", "")
dbutils.widgets.text("payload_storage", "")
dbutils.widgets.text("callback_behavior", "")
dbutils.widgets.text("catalog", "")
dbutils.widgets.text("schema_name", "")
dbutils.widgets.text("connection_id", "")
dbutils.widgets.text("connection_name", "")
dbutils.widgets.text("system_id", "")
dbutils.widgets.text("project_key", "")
dbutils.widgets.text("http_path", "")


from datetime import datetime
import requests
from pyspark.sql import SparkSession


spark = SparkSession.builder.getOrCreate()


_NOTEBOOK_STAGE_SEQUENCE = (
    ("parameters", "Cell 2 (widget and Spark initialization)"),
    ("profiling", "Cell 3 (profiling logic)"),
    ("payload_persistence", "Cell 4 (payload persistence and callbacks)"),
    ("metadata_helpers", "Cell 5 (metadata helper definitions)"),
    ("metadata_writer", "Cell 6 (metadata writer integration)"),
    ("finalization", "Cell 7 (final persistence and callbacks)"),
)
_NOTEBOOK_STAGE_LOOKUP = {label: (idx, hint) for idx, (label, hint) in enumerate(_NOTEBOOK_STAGE_SEQUENCE)}
_STAGE_SYMBOL_REQUIREMENTS = {
    "parameters": ("table_group_id", "profile_run_id", "dq_schema"),
    "profiling": ("MAX_COLUMNS_TO_PROFILE",),
    "payload_persistence": ("_resolve_payload_storage_mode", "_payload_storage_is_artifact"),
    "metadata_helpers": ("_persist_results_to_metadata",),
    "metadata_writer": ("_persist_profiling_metadata",),
}


def _ensure_notebook_stage(stage_label: str) -> None:
    if stage_label not in _NOTEBOOK_STAGE_LOOKUP:
        raise ValueError(f"Unknown notebook stage '{stage_label}'.")
    stage_index, stage_hint = _NOTEBOOK_STAGE_LOOKUP[stage_label]
    for prior_label, prior_hint in _NOTEBOOK_STAGE_SEQUENCE[:stage_index]:
        required_symbols = _STAGE_SYMBOL_REQUIREMENTS.get(prior_label, ())
        missing = [symbol for symbol in required_symbols if symbol not in globals()]
        if missing:
            missing_list = ", ".join(sorted(missing))
            raise RuntimeError(
                "Profiling notebook Cells 1-7 must run sequentially. "
                f"Run {prior_hint} before {stage_hint} (missing: {missing_list}).",
            )


table_group_id = dbutils.widgets.get("table_group_id")
profile_run_id = dbutils.widgets.get("profile_run_id")
dq_schema = (dbutils.widgets.get("data_quality_schema") or "").strip()
raw_payload_path = (dbutils.widgets.get("payload_path") or "").strip()
payload_path = raw_payload_path or None
payload_base_path = (dbutils.widgets.get("payload_base_path") or "").strip() or None
callback_url = (dbutils.widgets.get("callback_url") or "").strip() or None
callback_base_url = (dbutils.widgets.get("callback_base_url") or "").strip() or None
callback_token = (dbutils.widgets.get("callback_token") or "").strip() or None
connection_catalog = (dbutils.widgets.get("catalog") or "").strip()
connection_schema = (dbutils.widgets.get("schema_name") or "").strip()


if not table_group_id or not profile_run_id:
    raise ValueError("Required widgets missing: table_group_id/profile_run_id")
if not dq_schema:
    raise ValueError("Data quality schema widget is required for profiling runs.")


_ensure_notebook_stage("parameters")

In [ ]:
# Profile the tables registered for this table group and build the result payload.
import math
import uuid
from contextlib import suppress
from datetime import datetime, timezone
from functools import lru_cache
from typing import Any, Iterable

if "_ensure_notebook_stage" not in globals():
    raise RuntimeError("Profiling notebook Cells 1-6 must run sequentially; run Cell 2 before profiling.")

_ensure_notebook_stage("profiling")

import datetime as dt
import json

from pyspark.sql import DataFrame
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.utils import AnalysisException

from app.databricks_profiling import build_metadata_frames

MAX_COLUMNS_TO_PROFILE = 25
NULL_RATIO_ALERT_THRESHOLD = 0.5
HIGH_NULL_RATIO_THRESHOLD = 0.9
VALUE_DISTRIBUTION_LIMIT = 25
VALUE_DISTRIBUTION_DISTINCT_THRESHOLD = 1_000
VALUE_DISTRIBUTION_MAX_ROWS = 1_000_000
MAX_VALUE_DISPLAY_LENGTH = 256

PROFILE_COLUMN_FIELDS = [
    "profile_run_id",
    "schema_name",
    "table_name",
    "column_name",
    "qualified_name",
    "data_type",
    "general_type",
    "ordinal_position",
    "row_count",
    "null_count",
    "non_null_count",
    "distinct_count",
    "min_value",
    "max_value",
    "avg_value",
    "stddev_value",
    "median_value",
    "p95_value",
    "true_count",
    "false_count",
    "min_length",
    "max_length",
    "avg_length",
    "non_ascii_ratio",
    "min_date",
    "max_date",
    "date_span_days",
    "metrics_json",
    "generated_at",
]


def _now_utc() -> datetime:
    return datetime.now(timezone.utc)


def _qualify_metadata_table(table_name: str) -> str:
    table = (table_name or "").strip()
    if not table:
        raise ValueError("Metadata table name cannot be empty.")
    if connection_catalog:
        return f"`{connection_catalog}`.`{dq_schema}`.`{table}`"
    return f"`{dq_schema}`.`{table}`"


def _fetch_table_group_row() -> dict[str, Any]:
    table = _qualify_metadata_table("dq_table_groups")
    groups_df = spark.table(table).filter(F.col("table_group_id") == table_group_id).limit(1)
    rows = groups_df.collect()
    if not rows:
        raise ValueError(f"Table group {table_group_id} was not found in metadata tables.")
    return rows[0].asDict(recursive=True)


@lru_cache(maxsize=1)
def _fetch_tables_for_group() -> list[dict[str, Any]]:
    table = _qualify_metadata_table("dq_tables")
    tables_df = spark.table(table).filter(F.col("table_group_id") == table_group_id)
    return [row.asDict(recursive=True) for row in tables_df.collect()]


def _pattern_matches(value: str | None, pattern: str | None) -> bool:
    from fnmatch import fnmatchcase

    value = (value or "").strip().lower()
    pattern = (pattern or "").strip().lower()
    if not pattern:
        return True
    return fnmatchcase(value, pattern)


def _table_is_selected(table_entry: dict[str, Any], include_mask: str | None, exclude_mask: str | None) -> bool:
    logical_name = (table_entry.get("table_name") or "").strip()
    if include_mask and not _pattern_matches(logical_name, include_mask):
        return False
    if exclude_mask and _pattern_matches(logical_name, exclude_mask):
        return False
    return True


def _resolve_physical_reference(table_entry: dict[str, Any]) -> tuple[str, str]:
    schema_override = (table_entry.get("schema_name") or connection_schema or dq_schema or "").strip()
    schema_name = schema_override or dq_schema
    table_name = (table_entry.get("table_name") or "").strip()
    if not table_name:
        raise ValueError("Table entries must define table_name before profiling.")
    if connection_catalog:
        qualified = f"`{connection_catalog}`.`{schema_name}`.`{table_name}`"
    else:
        qualified = f"`{schema_name}`.`{table_name}`"
    return schema_name, qualified


def _is_numeric_type(data_type: T.DataType) -> bool:
    return isinstance(data_type, (
        T.ByteType,
        T.ShortType,
        T.IntegerType,
        T.LongType,
        T.FloatType,
        T.DoubleType,
        T.DecimalType,
    ))


def _is_boolean_type(data_type: T.DataType) -> bool:
    return isinstance(data_type, T.BooleanType)


def _is_date_type(data_type: T.DataType) -> bool:
    return isinstance(data_type, (T.DateType, T.TimestampType))


def _resolve_general_type(data_type: T.DataType) -> str:
    if _is_numeric_type(data_type):
        return "numeric"
    if _is_boolean_type(data_type):
        return "boolean"
    if isinstance(data_type, T.StringType):
        return "string"
    if _is_date_type(data_type):
        return "datetime"
    return data_type.simpleString()


def _stringify(value: Any) -> str | None:
    if value is None:
        return None
    if isinstance(value, (datetime, dt.date)):
        if isinstance(value, datetime) and value.tzinfo is None:
            value = value.replace(tzinfo=timezone.utc)
        return value.isoformat()
    text = str(value)
    return text[:MAX_VALUE_DISPLAY_LENGTH]


def _build_top_values(df: DataFrame, column: str, *, row_count: int, distinct_count: int | None) -> list[dict[str, Any]]:
    if distinct_count is not None and distinct_count > VALUE_DISTRIBUTION_DISTINCT_THRESHOLD:
        return []
    working_df = df
    if VALUE_DISTRIBUTION_MAX_ROWS and row_count > VALUE_DISTRIBUTION_MAX_ROWS:
        working_df = df.limit(VALUE_DISTRIBUTION_MAX_ROWS)
    agg_df = (
        working_df.groupBy(F.col(column)).count().orderBy(F.col("count").desc()).limit(VALUE_DISTRIBUTION_LIMIT)
    )
    rows = agg_df.collect()
    results: list[dict[str, Any]] = []
    for row in rows:
        row_dict = row.asDict(recursive=True)
        results.append(
            {
                "value": _stringify(row_dict.get(column)),
                "count": int(row_dict.get("count") or 0),
            }
        )
    return results


def _percentile_metrics(df: DataFrame, column: str) -> tuple[float | None, float | None]:
    try:
        percentiles = df.approxQuantile(column, [0.5, 0.95], 0.01)
    except Exception:
        return None, None
    median = percentiles[0] if len(percentiles) > 0 else None
    p95 = percentiles[1] if len(percentiles) > 1 else None
    return median, p95


def _profile_column(
    df: DataFrame,
    field: T.StructField,
    *,
    row_count: int,
    ordinal: int,
    schema_name: str,
    table_name: str,
) -> tuple[dict[str, Any], list[dict[str, Any]]]:
    column = field.name
    col_ref = F.col(column)

    aggregations = [
        F.count(col_ref).alias("non_null_count"),
        F.approx_count_distinct(col_ref, rsd=0.02).alias("distinct_count"),
        F.count(F.when(col_ref.isNull(), 1)).alias("null_count"),
    ]

    if _is_numeric_type(field.dataType):
        numeric_col = col_ref.cast("double")
        aggregations.extend(
            [
                F.min(numeric_col).alias("min_value"),
                F.max(numeric_col).alias("max_value"),
                F.avg(numeric_col).alias("avg_value"),
                F.stddev(numeric_col).alias("stddev_value"),
            ]
        )
    elif isinstance(field.dataType, T.StringType):
        length_col = F.length(col_ref)
        aggregations.extend(
            [
                F.min(col_ref).alias("min_value"),
                F.max(col_ref).alias("max_value"),
                F.min(length_col).alias("min_length"),
                F.max(length_col).alias("max_length"),
                F.avg(length_col).alias("avg_length"),
                F.avg(F.when(col_ref.rlike("[^\\x00-\\x7F]"), 1).otherwise(0)).alias("non_ascii_ratio"),
            ]
        )
    elif _is_date_type(field.dataType):
        date_col = col_ref.cast("timestamp")
        aggregations.extend(
            [
                F.min(date_col).alias("min_date"),
                F.max(date_col).alias("max_date"),
            ]
        )
    else:
        aggregations.extend(
            [
                F.min(col_ref).alias("min_value"),
                F.max(col_ref).alias("max_value"),
            ]
        )

    if _is_boolean_type(field.dataType):
        aggregations.extend(
            [
                F.sum(F.when(col_ref.isTrue(), 1).otherwise(0)).alias("true_count"),
                F.sum(F.when(col_ref.isFalse(), 1).otherwise(0)).alias("false_count"),
            ]
        )

    stats = df.agg(*aggregations).collect()[0]
    non_null_count = int(stats.non_null_count or 0)
    null_count = int(stats.null_count or 0)
    distinct_count = stats.distinct_count
    if distinct_count is not None:
        try:
            distinct_count = int(distinct_count)
        except (TypeError, ValueError):
            distinct_count = None

    median_value = None
    p95_value = None
    if _is_numeric_type(field.dataType):
        median_value, p95_value = _percentile_metrics(df, column)

    min_length = getattr(stats, "min_length", None)
    max_length = getattr(stats, "max_length", None)
    avg_length = getattr(stats, "avg_length", None)
    non_ascii_ratio = getattr(stats, "non_ascii_ratio", None)
    min_value = getattr(stats, "min_value", None)
    max_value = getattr(stats, "max_value", None)
    avg_value = getattr(stats, "avg_value", None)
    stddev_value = getattr(stats, "stddev_value", None)
    min_date = getattr(stats, "min_date", None)
    max_date = getattr(stats, "max_date", None)

    metrics = {
        "percentiles": {"median": median_value, "p95": p95_value},
        "top_values": [],
    }

    top_values = _build_top_values(df, column, row_count=row_count, distinct_count=distinct_count)
    if top_values:
        metrics["top_values"] = top_values

    metrics.update(
        {
            "row_count": row_count,
            "null_count": null_count,
            "non_null_count": non_null_count,
            "distinct_count": distinct_count,
            "min_length": min_length,
            "max_length": max_length,
            "avg_length": avg_length,
            "non_ascii_ratio": non_ascii_ratio,
            "min_value": _stringify(min_value),
            "max_value": _stringify(max_value),
            "avg_value": avg_value,
            "stddev_value": stddev_value,
            "min_date": _stringify(min_date),
            "max_date": _stringify(max_date),
        }
    )

    column_entry = {
        "column_name": column,
        "table_name": table_name,
        "schema_name": schema_name,
        "data_type": field.dataType.simpleString(),
        "general_type": _resolve_general_type(field.dataType),
        "ordinal_position": ordinal,
        "row_count": row_count,
        "null_count": null_count,
        "non_null_count": non_null_count,
        "distinct_count": distinct_count,
        "min_value": _stringify(min_value),
        "max_value": _stringify(max_value),
        "avg_value": avg_value,
        "stddev_value": stddev_value,
        "median_value": median_value,
        "p95_value": p95_value,
        "true_count": getattr(stats, "true_count", None),
        "false_count": getattr(stats, "false_count", None),
        "min_length": min_length,
        "max_length": max_length,
        "avg_length": avg_length,
        "non_ascii_ratio": non_ascii_ratio,
        "min_date": _stringify(min_date),
        "max_date": _stringify(max_date),
        "metrics": metrics,
    }

    if min_date and max_date:
        with suppress(Exception):
            delta_days = (max_date - min_date).days
            column_entry["date_span_days"] = delta_days
            metrics["date_span_days"] = delta_days

    column_anomalies: list[dict[str, Any]] = []
    null_ratio = float(null_count) / row_count if row_count else 0.0
    if null_ratio >= NULL_RATIO_ALERT_THRESHOLD:
        severity = "critical" if null_ratio >= HIGH_NULL_RATIO_THRESHOLD else "warning"
        column_anomalies.append(
            {
                "anomaly_type": "high_null_ratio",
                "severity": severity,
                "description": f"Null ratio {null_ratio:.2%} exceeds threshold",
                "column_name": column,
                "detected_at": _now_utc().isoformat(),
            }
        )

    if column_anomalies:
        column_entry["anomalies"] = column_anomalies

    return column_entry, column_anomalies


def _profile_table(
    table_entry: dict[str, Any],
    *,
    include_mask: str | None,
    exclude_mask: str | None,
) -> dict[str, Any] | None:
    if not _table_is_selected(table_entry, include_mask, exclude_mask):
        return None

    schema_name, qualified = _resolve_physical_reference(table_entry)
    table_name = table_entry.get("table_name")
    try:
        df = spark.table(qualified)
    except AnalysisException as exc:
        print(f"[profiling] Skipping table {qualified}: {exc}")
        return None

    row_count = df.count()
    if row_count == 0:
        print(f"[profiling] Table {qualified} returned zero rows; still capturing schema metadata.")

    columns = df.schema.fields[:MAX_COLUMNS_TO_PROFILE]
    profiled_columns: list[dict[str, Any]] = []
    anomalies: list[dict[str, Any]] = []

    for ordinal, field in enumerate(columns, start=1):
        try:
            column_entry, column_anomalies = _profile_column(
                df,
                field,
                row_count=row_count,
                ordinal=ordinal,
                schema_name=schema_name,
                table_name=table_name,
            )
        except Exception as exc:  # pragma: no cover - defensive guards for Spark failures
            print(f"[profiling] Failed to profile column {field.name} on {qualified}: {exc}")
            continue
        profiled_columns.append(column_entry)
        anomalies.extend(column_anomalies)

    table_payload = {
        "table_group_id": table_group_id,
        "table_id": table_entry.get("table_id"),
        "schema_name": schema_name,
        "table_name": table_name,
        "row_count": row_count,
        "column_count": len(profiled_columns),
        "columns": profiled_columns,
        "anomalies": anomalies,
        "metrics": {
            "record_count": row_count,
            "column_count": len(profiled_columns),
        },
    }
    return table_payload


def _build_profiling_payload() -> dict[str, Any]:
    group_row = _fetch_table_group_row()
    include_mask = group_row.get("profiling_include_mask")
    exclude_mask = group_row.get("profiling_exclude_mask")
    tables = []
    for entry in _fetch_tables_for_group():
        profiled = _profile_table(entry, include_mask=include_mask, exclude_mask=exclude_mask)
        if profiled:
            tables.append(profiled)

    if not tables:
        raise RuntimeError("No tables were profiled. Verify metadata selections and Databricks connectivity.")

    total_rows = sum(table.get("row_count") or 0 for table in tables)
    total_columns = sum(table.get("column_count", 0) for table in tables)
    anomaly_count = sum(len(table.get("anomalies") or []) for table in tables)

    started_at = _now_utc()
    completed_at = _now_utc()

    summary = {
        "status": "completed",
        "row_count": total_rows,
        "table_count": len(tables),
        "column_count": total_columns,
        "anomaly_count": anomaly_count,
        "profile_mode": "catalog",
        "profile_version": "catalog_v1",
        "started_at": started_at.isoformat(),
        "completed_at": completed_at.isoformat(),
    }

    payload = {
        "table_group_id": table_group_id,
        "profile_run_id": profile_run_id,
        "tables": tables,
        "summary": summary,
    }
    return payload


profiling_payload = _build_profiling_payload()
profiling_results = profiling_payload
profile_payload = profiling_payload
profile_results = profiling_payload
results_payload = profiling_payload

frames, frame_counts = build_metadata_frames(
    spark,
    profiling_payload,
    profile_run_id=profile_run_id,
    table_group_id=table_group_id,
    summary=profiling_payload.get("summary", {}),
)

for frame_name, df in frames.items():
    globals()[frame_name] = df

print(
    f"[profiling] Completed catalog-driven profiling for {len(profiling_payload['tables'])} tables "
    f"({profiling_payload['summary']['column_count']} columns)."
)


In [ ]:
# Persist payload artifacts and helper utilities


if "_ensure_notebook_stage" not in globals():
    raise RuntimeError("Profiling notebook Cells 1-6 must run sequentially; run prior cells before payload persistence.")


_ensure_notebook_stage("payload_persistence")


from datetime import datetime
import re
from contextlib import suppress
from functools import lru_cache
from typing import Optional


from pyspark.sql import functions as F
from pyspark.sql.utils import AnalysisException


DEFAULT_PRIVATE_PAYLOAD_ROOT = "dbfs:/tmp/conversioncentral/profiles"
DEFAULT_DRIVER_PAYLOAD_ROOT = "file:/databricks/driver/conversioncentral/profiles"
DEFAULT_PAYLOAD_STORAGE_MODE = "inline"
_VALID_PAYLOAD_STORAGE_MODES = {"inline", "artifact", "both"}
def _clean_widget_value(value: Optional[str]) -> str:
    return (value or "").strip()
def _resolve_payload_storage_mode() -> str:
    raw_value = _clean_widget_value(dbutils.widgets.get("payload_storage")).lower()
    if raw_value in _VALID_PAYLOAD_STORAGE_MODES:
        return raw_value
    if raw_value in {"inline_only", "inline_metadata"}:
        return "inline"
    if raw_value in {"artifact_only", "artifact_metadata"}:
        return "artifact"
    if payload_path:
        return "artifact"
    return DEFAULT_PAYLOAD_STORAGE_MODE
def _payload_storage_is_artifact(mode: str) -> bool:
    normalized = (mode or DEFAULT_PAYLOAD_STORAGE_MODE).strip().lower()
    return normalized in {"artifact", "both"}
DBFS_DISABLED_MESSAGES = ("public dbfs root is disabled", "access is denied")
DRIVER_DISABLED_MESSAGES = ("local filesystem access is forbidden", "workspacelocalfilesystem")
URI_SCHEME_PATTERN = re.compile(r"^[a-z][a-z0-9+.\-]*:/", re.IGNORECASE)
_DBFS_REDIRECT_NOTICE_EMITTED = False
_STORAGE_DISABLED_NOTICE_EMITTED = False

In [ ]:
# Column/value persistence helpers and overrides
import datetime as dt
from datetime import datetime, timezone
from contextlib import suppress
from typing import Any, Mapping

if "_ensure_notebook_stage" not in globals():
    raise RuntimeError("Profiling notebook Cells 1-7 must run sequentially; run earlier cells before defining metadata helpers.")


_ensure_notebook_stage("metadata_helpers")


def _escape_identifier(identifier: str) -> str:
    cleaned = (identifier or "").strip().replace("`", "")
    if not cleaned:
        raise ValueError("Metadata identifiers cannot be empty.")
    return f"`{cleaned}`"


def _metadata_schema_reference() -> str:
    if not dq_schema:
        raise ValueError("data_quality_schema widget must be set before resolving metadata tables.")
    catalog = (connection_catalog or "").strip()
    if catalog:
        return f"{_escape_identifier(catalog)}.{_escape_identifier(dq_schema)}"
    return _escape_identifier(dq_schema)


def _metadata_table(table_name: str) -> str:
    return f"{_metadata_schema_reference()}.{_escape_identifier(table_name)}"


def _first_non_empty(*values):
    for value in values:
        if isinstance(value, str):
            candidate = value.strip()
            if candidate:
                return candidate
        elif value is not None:
            return value
    return None


def _coerce_int(value):
    if value is None:
        return None
    if isinstance(value, bool):
        return int(value)
    if isinstance(value, int):
        return value
    if isinstance(value, float):
        if not math.isfinite(value):
            return None
        return int(round(value))
    if isinstance(value, str):
        candidate = value.strip().replace(",", "")
        if not candidate:
            return None
        try:
            if "." in candidate:
                return int(float(candidate))
            return int(candidate)
        except ValueError:
            return None
    return None


def _coerce_float(value):
    if value is None:
        return None
    if isinstance(value, bool):
        return float(value)
    if isinstance(value, (int, float)):
        numeric = float(value)
        if math.isfinite(numeric):
            return numeric
        return None
    if isinstance(value, str):
        candidate = value.strip().replace(",", "")
        if not candidate:
            return None
        try:
            numeric = float(candidate)
        except ValueError:
            return None
        return numeric if math.isfinite(numeric) else None
    return None


def _sql_literal(value) -> str:
    if value is None:
        return "NULL"
    if isinstance(value, datetime):
        if value.tzinfo is None:
            value = value.replace(tzinfo=timezone.utc)
        else:
            value = value.astimezone(timezone.utc)
        return f"'{value.strftime('%Y-%m-%d %H:%M:%S')}'"
    text = str(value).replace("'", "''")
    return f"'{text}'"


def _sql_number(value) -> str:
    if value is None:
        return "NULL"
    return str(value)


def _sql_literal_tuple(values) -> str:
    inner = ", ".join(_sql_literal(value) for value in values)
    return f"({inner})"


def _sql_literal_set(values) -> str:
    if not values:
        return "(NULL)"
    return f"({', '.join(_sql_literal(value) for value in values)})"


def _coerce_timestamp_value(value) -> datetime | None:
    if isinstance(value, datetime):
        if value.tzinfo is None:
            return value.replace(tzinfo=timezone.utc)
        return value.astimezone(timezone.utc)
    if isinstance(value, (int, float)):
        numeric = float(value)
        if abs(numeric) > 1_000_000_000_000:
            numeric /= 1000.0
        with suppress(Exception):
            return datetime.fromtimestamp(numeric, tz=timezone.utc)
        return None
    if isinstance(value, str):
        text = value.strip()
        if not text:
            return None
        normalized = text[:-1] + "+00:00" if text.endswith("Z") else text
        with suppress(ValueError):
            parsed = datetime.fromisoformat(normalized)
            if parsed.tzinfo is None:
                return parsed.replace(tzinfo=timezone.utc)
            return parsed.astimezone(timezone.utc)
        for fmt in ("%Y-%m-%d %H:%M:%S", "%Y-%m-%dT%H:%M:%S"):
            with suppress(ValueError):
                parsed = datetime.strptime(normalized, fmt)
                return parsed.replace(tzinfo=timezone.utc)
    return None


def _resolve_databricks_run_id() -> str | None:
    with suppress(Exception):
        value = spark.conf.get("spark.databricks.job.runId")
        if value:
            return str(value)
    with suppress(Exception):
        ctx = dbutils.notebook.entry_point.getDbutils().notebook().getContext()
        run_id = ctx.runId().get()
        if run_id:
            return str(run_id)
    with suppress(Exception):
        task_run_id = dbutils.jobs.taskRunId()
        if task_run_id:
            return str(task_run_id)
    return None


def _extract_profile_summary(payload: Any) -> Mapping[str, Any]:
    if isinstance(payload, Mapping):
        for key in ("summary", "profile_summary", "profileSummary", "metadata", "run"):
            nested = payload.get(key)
            if isinstance(nested, Mapping):
                return nested
        return payload
    if isinstance(payload, list):
        for item in payload:
            if isinstance(item, Mapping):
                return item
    return {}


def _persist_results_to_metadata(results_payload, payload_location):
    if not table_group_id:
        raise ValueError("table_group_id must be defined before persisting metadata.")
    if not profile_run_id:
        raise ValueError("profile_run_id must be defined before persisting metadata.")

    summary = _extract_profile_summary(results_payload) if results_payload is not None else {}
    status = _first_non_empty(summary.get("status"), summary.get("state"), "completed")
    started_at = _coerce_timestamp_value(summary.get("started_at") or summary.get("startedAt"))
    completed_at = _coerce_timestamp_value(summary.get("completed_at") or summary.get("completedAt"))
    row_count = _coerce_int(
        summary.get("row_count")
        or summary.get("rowCount")
        or summary.get("rows")
        or summary.get("total_rows")
        or summary.get("totalRows")
    )
    anomaly_count = _coerce_int(summary.get("anomaly_count") or summary.get("anomalyCount"))
    if anomaly_count is None:
        anomalies = summary.get("anomalies")
        if isinstance(anomalies, (list, tuple)):
            anomaly_count = len(anomalies)

    if started_at is None:
        started_at = datetime.now(timezone.utc)
    if completed_at is None:
        completed_at = datetime.now(timezone.utc)

    payload_ref = _first_non_empty(payload_location, summary.get("payload_path"), summary.get("payloadPath"))
    profiles_table = _metadata_table("dq_profiles")
    profile_literal = _sql_literal(profile_run_id)
    spark.sql(
        f"DELETE FROM {profiles_table} WHERE {_escape_identifier('profile_run_id')} = {profile_literal}"
    )

    columns = (
        "profile_run_id",
        "table_group_id",
        "status",
        "started_at",
        "completed_at",
        "row_count",
        "anomaly_count",
        "payload_path",
        "databricks_run_id",
    )
    values = [
        _sql_literal(profile_run_id),
        _sql_literal(table_group_id),
        _sql_literal(status),
        _sql_literal(started_at),
        _sql_literal(completed_at),
        _sql_number(row_count),
        _sql_number(anomaly_count),
        _sql_literal(payload_ref),
        _sql_literal(_resolve_databricks_run_id()),
    ]
    columns_sql = ", ".join(_escape_identifier(column) for column in columns)
    values_sql = ", ".join(values)
    spark.sql(f"INSERT INTO {profiles_table} ({columns_sql}) VALUES ({values_sql})")

    ref_label = payload_ref or "inline"
    print(
        f"Persisted metadata for profile run {profile_run_id} with status '{status}' and payload reference {ref_label}."
    )

In [ ]:
# Metadata persistence refinements to avoid duplicate dq_profiles rows
if "_persist_results_to_metadata" not in globals():
    raise RuntimeError(
        "Profiling notebook metadata helpers must run before overriding _persist_results_to_metadata."
    )


def _persist_results_to_metadata(results_payload, payload_location):
    """Persist profiling run metadata using a MERGE to update the existing row."""
    if not table_group_id:
        raise ValueError("table_group_id must be defined before persisting metadata.")
    if not profile_run_id:
        raise ValueError("profile_run_id must be defined before persisting metadata.")

    summary = _extract_profile_summary(results_payload) if results_payload is not None else {}
    status = _first_non_empty(summary.get("status"), summary.get("state"), "completed")
    started_at = _coerce_timestamp_value(summary.get("started_at") or summary.get("startedAt"))
    completed_at = _coerce_timestamp_value(summary.get("completed_at") or summary.get("completedAt"))
    row_count = _coerce_int(
        summary.get("row_count")
        or summary.get("rowCount")
        or summary.get("rows")
        or summary.get("total_rows")
        or summary.get("totalRows")
    )
    anomaly_count = _coerce_int(summary.get("anomaly_count") or summary.get("anomalyCount"))
    if anomaly_count is None:
        anomalies = summary.get("anomalies")
        if isinstance(anomalies, (list, tuple)):
            anomaly_count = len(anomalies)

    if started_at is None:
        started_at = datetime.now(timezone.utc)
    if completed_at is None:
        completed_at = datetime.now(timezone.utc)

    payload_ref = _first_non_empty(payload_location, summary.get("payload_path"), summary.get("payloadPath"))
    profiles_table = _metadata_table("dq_profiles")

    supported_columns = set()
    try:
        schema = spark.table(profiles_table).schema
        supported_columns = {field.name for field in schema}
    except Exception as exc:  # pragma: no cover - descriptive logging only
        print(f"[metadata] Unable to inspect schema for {profiles_table}: {exc}")

    if "payload_path" not in supported_columns:
        with suppress(Exception):
            spark.sql(f"ALTER TABLE {profiles_table} ADD COLUMNS (payload_path STRING)")
            try:
                schema = spark.table(profiles_table).schema
                supported_columns = {field.name for field in schema}
            except Exception as schema_exc:  # pragma: no cover - descriptive logging only
                print(f"[metadata] Unable to refresh schema for {profiles_table}: {schema_exc}")

    def _column_supported(column: str) -> bool:
        return not supported_columns or column in supported_columns

    column_specs = [
        ("profile_run_id", _sql_literal(profile_run_id)),
        ("table_group_id", _sql_literal(table_group_id)),
        ("status", _sql_literal(status)),
        ("started_at", _sql_literal(started_at)),
        ("completed_at", _sql_literal(completed_at)),
        ("row_count", _sql_number(row_count)),
        ("anomaly_count", _sql_number(anomaly_count)),
        ("payload_path", _sql_literal(payload_ref)),
        ("databricks_run_id", _sql_literal(_resolve_databricks_run_id())),
    ]

    filtered_specs = []
    skipped_columns = []
    for column, value in column_specs:
        if column != "profile_run_id" and not _column_supported(column):
            skipped_columns.append(column)
            continue
        filtered_specs.append((column, value))

    if len(filtered_specs) < 2:
        raise RuntimeError(
            "Profiling metadata merge could not map any target columns; verify dq_profiles schema permissions."
        )

    column_names = [name for name, _ in filtered_specs]
    columns_sql = ", ".join(_escape_identifier(name) for name in column_names)
    updates_sql = ", ".join(
        f"target.{_escape_identifier(name)} = source.{_escape_identifier(name)}"
        for name in column_names
        if name != "profile_run_id"
    )
    source_select_sql = ", ".join(
        f"{value} AS {_escape_identifier(name)}" for name, value in filtered_specs
    )
    insert_values_sql = ", ".join(
        f"source.{_escape_identifier(name)}" for name in column_names
    )

    merge_sql = (
        f"MERGE INTO {profiles_table} AS target "
        f"USING (SELECT {source_select_sql}) AS source "
        f"ON target.{_escape_identifier('profile_run_id')} = source.{_escape_identifier('profile_run_id')} "
    )
    if updates_sql:
        merge_sql += f"WHEN MATCHED THEN UPDATE SET {updates_sql} "
    merge_sql += f"WHEN NOT MATCHED THEN INSERT ({columns_sql}) VALUES ({insert_values_sql})"

    spark.sql(merge_sql)

    if skipped_columns:
        skipped = ", ".join(sorted(set(skipped_columns)))
        print(f"[metadata] Skipped unsupported columns during dq_profiles merge: {skipped}")

    ref_label = payload_ref or "inline"
    print(
        f"Persisted metadata for profile run {profile_run_id} with status '{status}' and payload reference {ref_label}."
    )


In [ ]:
# Metadata writer integration
import sys
from pathlib import Path
from typing import Any, Sequence


if "_ensure_notebook_stage" not in globals():
    raise RuntimeError("Profiling notebook Cells 1-7 must run sequentially; run prior cells before metadata writer integration.")


_ensure_notebook_stage("metadata_writer")


_PROFILE_METADATA_WRITER: "ProfilingMetadataWriter | None" = None
_METADATA_FRAME_SPECS: Sequence[dict[str, Any]] = (
    {
        "frame_name": "profile_results_df",
        "table_name": "dq_profile_results",
        "key_columns": ["profile_run_id", "table_name", "column_name"],
    },
    {
        "frame_name": "profile_columns_df",
        "table_name": "dq_profile_columns",
        "key_columns": ["profile_run_id", "table_name", "column_name"],
    },
    {
        "frame_name": "profile_column_values_df",
        "table_name": "dq_profile_column_values",
        "key_columns": ["profile_run_id", "table_name", "column_name", "value_hash"],
    },
    {
        "frame_name": "profile_anomalies_df",
        "table_name": "dq_profile_anomaly_results",
        "key_columns": ["profile_run_id", "table_name", "column_name", "anomaly_type_id"],
    },
    {
        "frame_name": "table_characteristics_df",
        "table_name": "dq_data_table_chars",
        "key_columns": ["table_id"],
    },
    {
        "frame_name": "column_characteristics_df",
        "table_name": "dq_data_column_chars",
        "key_columns": ["column_id"],
    },
)


def _ensure_repo_root_on_path() -> Path | None:
    candidates = [Path.cwd(), *Path.cwd().parents]
    for candidate in candidates:
        app_dir = candidate / "app"
        if app_dir.exists():
            candidate_str = str(candidate)
            if candidate_str not in sys.path:
                sys.path.insert(0, candidate_str)
            return candidate
    return None


def _resolve_metadata_writer() -> "ProfilingMetadataWriter":
    global _PROFILE_METADATA_WRITER
    if _PROFILE_METADATA_WRITER is not None:
        return _PROFILE_METADATA_WRITER

    repo_root = _ensure_repo_root_on_path()
    if repo_root is None:
        raise RuntimeError("Unable to locate repo root containing the 'app' package. Ensure the notebook runs from a Databricks Repo checkout.")

    from app.databricks_profiling import ProfilingMetadataWriter

    _PROFILE_METADATA_WRITER = ProfilingMetadataWriter(
        spark,
        schema=dq_schema,
        catalog=connection_catalog or None,
        profile_run_id=profile_run_id,
    )
    return _PROFILE_METADATA_WRITER


def _merge_metadata_dataframe(
    df,
    *,
    target_table: str,
    key_columns: Sequence[str],
    update_columns: Sequence[str] | None = None,
) -> int:
    if df is None:
        return 0
    writer = _resolve_metadata_writer()
    return writer.merge_dataframe(
        df,
        target_table=target_table,
        key_columns=key_columns,
        update_columns=update_columns,
    )


def _autofill_missing_metadata_frames() -> dict[str, int]:
    if "_autopopulate_metadata_frames" not in globals():
        return {}
    missing_frames = [
        spec["frame_name"]
        for spec in _METADATA_FRAME_SPECS
        if spec["frame_name"] not in globals()
    ]
    if not missing_frames:
        return {}
    created_counts = _autopopulate_metadata_frames()
    for frame_name in missing_frames:
        if frame_name in created_counts:
            print(
                f"[metadata] Auto-built DataFrame '{frame_name}' with {created_counts[frame_name]} rows before persistence."
            )
    return created_counts


def _persist_profiling_metadata() -> dict[str, int]:
    autofill_counts = _autofill_missing_metadata_frames()
    summary: dict[str, int] = {}
    for spec in _METADATA_FRAME_SPECS:
        frame_name = spec["frame_name"]
        target_table = spec["table_name"]
        key_columns = spec["key_columns"]
        df = globals().get(frame_name)
        if df is None:
            summary[frame_name] = 0
            if frame_name not in autofill_counts:
                print(f"[metadata] DataFrame '{frame_name}' not defined; skipping {target_table}.")
            continue
        rows = _merge_metadata_dataframe(
            df,
            target_table=target_table,
            key_columns=key_columns,
            update_columns=spec.get("update_columns"),
        )
        summary[frame_name] = rows
        print(f"[metadata] Persisted {rows} rows from {frame_name} into {target_table}.")
    return summary

In [ ]:
# Metadata payload normalization and DataFrame bootstrap
import json

if "_ensure_notebook_stage" not in globals():
    raise RuntimeError(
        "Profiling notebook Cells 1-7 must run sequentially; run prior cells before metadata frame bootstrap.",
    )

_ensure_notebook_stage("metadata_writer")

_METADATA_PAYLOAD_CANDIDATES = (
    "profiling_payload",
    "results_payload",
    "profile_payload",
    "profiling_results",
    "profile_results",
    "results",
)


def _resolve_metadata_payload_value():
    for name in _METADATA_PAYLOAD_CANDIDATES:
        if name in globals():
            value = globals()[name]
            if value is not None:
                return value
    return None


def _normalize_metadata_payload(value):
    if value is None:
        return None
    if isinstance(value, str):
        text = value.strip()
        if not text:
            return None
        if text.startswith("{") or text.startswith("["):
            try:
                return json.loads(text)
            except json.JSONDecodeError:
                return None
        return None
    return value


def _autopopulate_metadata_frames() -> dict[str, int]:
    missing_frames = [spec["frame_name"] for spec in _METADATA_FRAME_SPECS if spec["frame_name"] not in globals()]
    if not missing_frames:
        return {}

    payload = _normalize_metadata_payload(_resolve_metadata_payload_value())
    if payload is None:
        print("[metadata] Profiling payload unavailable; skipping automatic DataFrame creation.")
        return {}

    repo_root = _ensure_repo_root_on_path()
    if repo_root is None:
        print("[metadata] Repo root not found; skipping automatic DataFrame creation.")
        return {}

    try:
        from app.databricks_profiling import build_metadata_frames
    except Exception as exc:  # pragma: no cover - defensive import guard
        print(f"[metadata] Unable to import profiling frame builder: {exc}")
        return {}

    summary = _extract_profile_summary(payload) if "_extract_profile_summary" in globals() else {}
    try:
        frames, counts = build_metadata_frames(
            spark,
            payload,
            profile_run_id=profile_run_id,
            table_group_id=table_group_id,
            summary=summary,
        )
    except Exception as exc:  # pragma: no cover - builder errors during notebook execution
        print(f"[metadata] Failed to build profiling DataFrames: {exc}")
        return {}

    created_counts: dict[str, int] = {}
    for name in missing_frames:
        df = frames.get(name)
        if df is None:
            continue
        globals()[name] = df
        created_counts[name] = counts.get(name, 0)

    if not created_counts:
        print("[metadata] Profiling payload parsed but produced no rows for the requested frames.")
    return created_counts


_METADATA_AUTOFILL_COUNTS = _autopopulate_metadata_frames()
if _METADATA_AUTOFILL_COUNTS:
    for frame_name, row_count in sorted(_METADATA_AUTOFILL_COUNTS.items()):
        print(f"[metadata] Auto-built DataFrame '{frame_name}' with {row_count} rows from profiling payload.")

In [ ]:
# Final metadata persistence summary
import json
from typing import Any

if "_ensure_notebook_stage" not in globals():
    raise RuntimeError("Profiling notebook Cells 1-7 must run sequentially; run prior cells before finalization.")


_ensure_notebook_stage("finalization")


def _resolve_results_payload() -> Any:
    """Pick the richest profiling payload produced by earlier cells."""
    for name in (
        "results_payload",
        "profiling_payload",
        "profile_payload",
        "profiling_results",
        "profile_results",
        "results",
    ):
        if name in globals():
            return globals()[name]
    return None


def _resolve_payload_reference() -> str | None:
    for candidate in (
        globals().get("persisted_payload_path"),
        globals().get("payload_reference"),
        globals().get("payload_location"),
        globals().get("payload_artifact_path"),
        payload_path,
        raw_payload_path,
    ):
        if isinstance(candidate, str):
            normalized = candidate.strip()
            if normalized:
                return normalized
    return None


resolved_storage_mode = _resolve_payload_storage_mode()
results_payload = _resolve_results_payload()
results_summary = _extract_profile_summary(results_payload) if results_payload is not None else {}
status = _first_non_empty(results_summary.get("status"), results_summary.get("state"), "completed")
payload_reference = _resolve_payload_reference()
if not payload_reference and _payload_storage_is_artifact(resolved_storage_mode):
    payload_reference = payload_path

_persist_results_to_metadata(results_payload, payload_reference)

metadata_write_counts: dict[str, Any] = {}
if "_persist_profiling_metadata" in globals():
    try:
        metadata_write_counts = _persist_profiling_metadata()
    except Exception as exc:  # pragma: no cover - defensive logging for notebook runtime
        metadata_write_counts = {"error": str(exc)}
        print(f"[metadata] Failed to persist profiling metadata: {exc}")
else:
    print("[metadata] Metadata writer helper not defined; skipping DataFrame persistence.")

FINALIZATION_CONTEXT = {
    "profile_run_id": profile_run_id,
    "table_group_id": table_group_id,
    "status": status,
    "payload_reference": payload_reference,
    "payload_storage_mode": resolved_storage_mode,
    "metadata_schema": dq_schema,
    "databricks_run_id": _resolve_databricks_run_id(),
    "results_summary": results_summary,
    "metadata_write_counts": metadata_write_counts,
}

print(json.dumps(FINALIZATION_CONTEXT, indent=2, sort_keys=True))